In [86]:
import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.layers import Input, LSTM, Attention, Embedding, Dense, Concatenate, TimeDistributed   #Layers required to implement the model
# Try out

In [87]:

import numpy as np   #Package for scientific computing and dealing with arrays
import pandas as pd  #Package providing fast, flexible and expressive data structures
import re            #re stands for RegularExpression providing full support for Perl-like Regular Expressions in Python
from bs4 import BeautifulSoup   #Package for pulling data out of HTML and XML files
from keras.preprocessing.sequence import pad_sequences  #For Padding the seqences to same length
from nltk.corpus import stopwords   #For removing filler words
from tensorflow.keras.layers import Input, LSTM, Attention, Embedding, Dense, Concatenate, TimeDistributed   #Layers required to implement the model
from tensorflow.keras.models import Model  #Helps in grouping the layers into an object with training and inference features
from tensorflow.keras.callbacks import EarlyStopping  #Allows training the model on large no. of training epochs & stop once the performance stops improving on validation dataset
from os import listdir
import string

In [88]:

# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# split a document into news story and highlights
def split_story(doc):
	# find first highlight
	index = doc.find('@highlight')
	# split into story and highlights
	story, highlights = doc[:index], doc[index:].split('@highlight')
	# strip extra white space around each highlight
	highlights = [h.strip() for h in highlights if len(h) > 0]
	return story, highlights

# load all stories in a directory
def load_stories(directory):
	stories = list()
	for name in listdir(directory):
		filename = directory + '/' + name
		# load document
		doc = load_doc(filename)
		# split into story and highlights
		story, highlights = split_story(doc)
		# store
		stories.append({'story':story, 'highlights':highlights})
	return stories

# clean a list of lines
def clean_lines(lines):
	cleaned = list()
	# prepare a translation table to remove punctuation
	table = str.maketrans('', '', string.punctuation)
	for line in lines:
		# strip source cnn office if it exists
		index = line.find('(CNN) -- ')
		if index > -1:
			line = line[index+len('(CNN)'):]
		# tokenize on white space
		line = line.split()
		# convert to lower case
		line = [word.lower() for word in line]
		# remove punctuation from each token
		line = [w.translate(table) for w in line]
		# remove tokens with numbers in them
		line = [word for word in line if word.isalpha()]
		# store as string
		cleaned.append(' '.join(line))
	# remove empty strings
	cleaned = [c for c in cleaned if len(c) > 0]
	return cleaned

# load stories
directory = 'cnn/'
stories = load_stories(directory)
print('Loaded Stories %d' % len(stories))

# clean stories
for example in stories:
	example['story'] = clean_lines(example['story'].split('\n'))
	example['highlights'] = clean_lines(example['highlights'])

Loaded Stories 23


In [89]:
# save to file
from pickle import dump
dump(stories, open('cnn_dataset.pkl', 'wb'))

In [90]:
from pickle import load

# load from file
stories = load(open('cnn_dataset.pkl', 'rb'))
print('Loaded Stories %d' % len(stories))

Loaded Stories 23


In [91]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import RepeatVector


In [92]:
(stories[0]['story'])

['after a weekend of intense investigation authorities are piecing together more details about friday s fatal shooting at los angeles international airport including the suspect s behavior earlier in the week and a warning from his family that may have come minutes too late',
 'officers sent to check on paul ciancia s welfare arrived at his apartment less than an hour after the shooting started police said monday',
 'here is a rundown to get you up to speed',
 'the suspect',
 'ciancia of los angeles is charged with murder of a federal officer and commission of violence in an international airport',
 'he was shot by officers friday and was in critical condition at ronald reagan ucla medical center on sunday',
 'a source said ciancia was unable to speak to investigators',
 'clues about a motive',
 'attorney general eric holder said monday that more investigation is necessary to uncover a motive for the attack',
 'but a note found on ciancia indicated that he wanted to kill transportation

In [93]:
#Preprocessing

#This the dictionary used for expanding contractions
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [94]:
#Text Cleaning
import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words('english')) 
def text_cleaner(text,num):
    newString = text.lower()  #converts all uppercase characters in the string into lowercase characters and returns it
    newString = BeautifulSoup(newString, "lxml").text #parses the string into an lxml.html 
    newString = re.sub(r'\([^)]*\)', '', newString) #used to replace a string that matches a regular expression instead of perfect match
    newString = re.sub('"','', newString)           
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")]) #for expanding contractions using the contraction_mapping dictionary    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    if(num==0): 
      tokens = [w for w in newString.split() if not w in stop_words]  #converting the strings into tokens
    else :
      tokens = newString.split()
    long_words=[]
    for i in tokens:
        if len(i)>1:                  #removing short words
            long_words.append(i)   
    return (" ".join(long_words)).strip()



[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1108)>


In [95]:
ss  = edict()
ss["idan"] = 1
ss["idan"]+1
ss

{'idan': 1}

In [96]:
from easydict import EasyDict as edict

In [97]:
#Calling the function
x_vocab = edict()
y_vocab = edict()

article_word_count = []
abstract_word_count = []

cleaned_text = []
highlights = []

max_ar_length = 0
max_high_lengh = 0

for file in stories:
    t = file['story']
    story =""
    highligh= ""
    h = file['highlights']
    for line in t:
        story= story+line
        for word in line: 
            if word not in x_vocab.keys():
                x_vocab[word] =1
            else: 
                x_vocab[word]+=1

    if len(story.split())> max_ar_length: max_ar_length =len(story.split())
    article_word_count.append(len(story.split()))
    cleaned_text.append(text_cleaner(story,0))
    for line in h:
        highligh= highligh+line
        if word not in y_vocab.keys():
            y_vocab[word] =1
        else: 
            y_vocab[word]+=1

    if len(highligh.split())> max_high_lengh: max_high_lengh =len(highligh.split())
    abstract_word_count.append(len(highligh.split()))
    highlights.append(highligh)

In [98]:
# print summer: 
print(f" Max length sequence story: {max_ar_length}")
print(f" Max length sequence story: {max_high_lengh}")



Max length sequence story: 1329
 Max length sequence story: 56


In [99]:
type(len(set(cleaned_text[1].split())))

int

In [100]:
# TODO: some words are not properly split; maybe beginning and ending of sentences?
cleaned_text[0]

'weekend intense investigation authorities piecing together details friday fatal shooting los angeles international airport including suspect behavior earlier week warning family may come minutes lateofficers sent check paul ciancia welfare arrived apartment less hour shooting started police said mondayhere rundown get speedthe suspectciancia los angeles charged murder federal officer commission violence international airporthe shot officers friday critical condition ronald reagan ucla medical center sundaya source said ciancia unable speak investigatorsclues motiveattorney general eric holder said monday investigation necessary uncover motive attackbut note found ciancia indicated wanted kill transportation security administration employees instill fear suspect called agents traitorous minds fbi special agent charge david bowdich saidaccording someone knew ciancia three roommates well ciancia began asking ride airport days shooting claimed needed fly new jersey help sick father never 

In [101]:
highlights[0] = "sos " + highlights[0] + " eos"
highlights[0]

'sos an injured officer says he was shot while trying to help an elderly man move to safetylapd provides timeline for welfare check at suspect s homesuspect paul ciancia is in critical condition and could face the death penaltypolice responded to family concern arrived at ciancia s apartment shortly after he left eos'

In [102]:
X_tokenizer = Tokenizer(num_words = len(set(cleaned_text[0].split()))+1)
X_tokenizer.fit_on_texts(cleaned_text[0].split())


In [103]:
X_tokenizer.word_counts

OrderedDict([('weekend', 1),
             ('intense', 1),
             ('investigation', 3),
             ('authorities', 2),
             ('piecing', 1),
             ('together', 1),
             ('details', 1),
             ('friday', 6),
             ('fatal', 1),
             ('shooting', 8),
             ('los', 5),
             ('angeles', 6),
             ('international', 3),
             ('airport', 8),
             ('including', 2),
             ('suspect', 3),
             ('behavior', 1),
             ('earlier', 2),
             ('week', 1),
             ('warning', 1),
             ('family', 12),
             ('may', 2),
             ('come', 1),
             ('minutes', 4),
             ('lateofficers', 1),
             ('sent', 1),
             ('check', 2),
             ('paul', 2),
             ('ciancia', 20),
             ('welfare', 1),
             ('arrived', 4),
             ('apartment', 3),
             ('less', 1),
             ('hour', 1),
             ('s

In [104]:
# TODO: why 90? this will probably fuck up the embedding layer because we only have 51 words in the abstract and the embedding thinks our vocabulary has size 90
Y_tokenizer = Tokenizer(num_words = len(set(highlights[0].split())) + 1)
Y_tokenizer.fit_on_texts(highlights[0].split())
Y_voc = Y_tokenizer.num_words

In [105]:
Y_tokenizer.word_counts

OrderedDict([('sos', 1),
             ('an', 2),
             ('injured', 1),
             ('officer', 1),
             ('says', 1),
             ('he', 2),
             ('was', 1),
             ('shot', 1),
             ('while', 1),
             ('trying', 1),
             ('to', 3),
             ('help', 1),
             ('elderly', 1),
             ('man', 1),
             ('move', 1),
             ('safetylapd', 1),
             ('provides', 1),
             ('timeline', 1),
             ('for', 1),
             ('welfare', 1),
             ('check', 1),
             ('at', 2),
             ('suspect', 1),
             ('s', 2),
             ('homesuspect', 1),
             ('paul', 1),
             ('ciancia', 2),
             ('is', 1),
             ('in', 1),
             ('critical', 1),
             ('condition', 1),
             ('and', 1),
             ('could', 1),
             ('face', 1),
             ('the', 1),
             ('death', 1),
             ('penaltypolice', 

In [106]:
def word_to_idx(data: list) -> dict:
    """
    Function that maps the data and return a dictionary of words corresponding to their index
    it gets a list
    return:
        dict 1 idx to word
        dict 2 word to idx
    """
    total_letters = [letters for sublist in data for subsublist in sublist for letters in subsublist]
    unique_letters =set(total_letters)
    total_words = [word.replace(',','') for sublist in data for subsublist in sublist for word in subsublist.split()]
    unique_words =list(set(total_words))

    w_2_i = {unique_words[i]:i for i in range(len(unique_words))}
    i_2_w= {i: unique_words[i] for i in range(len(unique_words))}
    print(w_2_i)
    input()
    return (w_2_i, i_2_w)

In [107]:
X_train= X_tokenizer.texts_to_sequences(cleaned_text[0].split()) 
y_train  = Y_tokenizer.texts_to_sequences(highlights[0].split())

In [108]:
len(X_train)

735

In [109]:
len(y_train)

53

In [110]:
X_tokenizer.num_words

491

In [111]:
Y_tokenizer.num_words

47

In [112]:
X_train

[[109],
 [110],
 [23],
 [42],
 [111],
 [112],
 [113],
 [9],
 [114],
 [5],
 [14],
 [10],
 [24],
 [6],
 [43],
 [25],
 [115],
 [44],
 [116],
 [117],
 [3],
 [45],
 [118],
 [17],
 [119],
 [120],
 [46],
 [47],
 [1],
 [121],
 [18],
 [26],
 [122],
 [123],
 [5],
 [124],
 [4],
 [2],
 [125],
 [126],
 [127],
 [128],
 [129],
 [14],
 [10],
 [130],
 [131],
 [132],
 [19],
 [133],
 [134],
 [24],
 [135],
 [11],
 [8],
 [9],
 [136],
 [27],
 [137],
 [138],
 [139],
 [140],
 [141],
 [142],
 [28],
 [2],
 [1],
 [143],
 [144],
 [145],
 [146],
 [48],
 [147],
 [148],
 [2],
 [12],
 [23],
 [149],
 [150],
 [151],
 [152],
 [153],
 [154],
 [1],
 [155],
 [156],
 [157],
 [49],
 [7],
 [29],
 [50],
 [158],
 [159],
 [25],
 [30],
 [160],
 [161],
 [162],
 [31],
 [163],
 [164],
 [165],
 [166],
 [167],
 [168],
 [169],
 [51],
 [1],
 [170],
 [171],
 [52],
 [1],
 [53],
 [32],
 [54],
 [6],
 [55],
 [5],
 [172],
 [33],
 [173],
 [34],
 [56],
 [174],
 [175],
 [57],
 [176],
 [2],
 [177],
 [33],
 [178],
 [28],
 [179],
 [9],
 [1],
 [180]

In [113]:
y_train

[[7],
 [2],
 [8],
 [9],
 [10],
 [3],
 [11],
 [12],
 [13],
 [14],
 [1],
 [15],
 [2],
 [16],
 [17],
 [18],
 [1],
 [19],
 [20],
 [21],
 [22],
 [23],
 [24],
 [4],
 [25],
 [5],
 [26],
 [27],
 [6],
 [28],
 [29],
 [30],
 [31],
 [32],
 [33],
 [34],
 [35],
 [36],
 [37],
 [38],
 [1],
 [39],
 [40],
 [41],
 [4],
 [6],
 [5],
 [42],
 [43],
 [44],
 [3],
 [45],
 [46]]

In [114]:
X_voc = X_tokenizer.num_words

In [115]:
type(highlights[1])

str

In [116]:
y = np.array(y_train)
x = np.array(X_train)

In [117]:
y.shape

(53, 1)

In [118]:
x.shape

(735, 1)

In [119]:
x = x.reshape(1,-1)
y = y.reshape(1,-1)


In [120]:
x.shape

(1, 735)

In [121]:
y.shape

(1, 53)

In [122]:
x

array([[109, 110,  23,  42, 111, 112, 113,   9, 114,   5,  14,  10,  24,
          6,  43,  25, 115,  44, 116, 117,   3,  45, 118,  17, 119, 120,
         46,  47,   1, 121,  18,  26, 122, 123,   5, 124,   4,   2, 125,
        126, 127, 128, 129,  14,  10, 130, 131, 132,  19, 133, 134,  24,
        135,  11,   8,   9, 136,  27, 137, 138, 139, 140, 141, 142,  28,
          2,   1, 143, 144, 145, 146,  48, 147, 148,   2,  12,  23, 149,
        150, 151, 152, 153, 154,   1, 155, 156, 157,  49,   7,  29,  50,
        158, 159,  25,  30, 160, 161, 162,  31, 163, 164, 165, 166, 167,
        168, 169,  51,   1, 170, 171,  52,   1,  53,  32,  54,   6,  55,
          5, 172,  33, 173,  34,  56, 174, 175,  57, 176,   2, 177,  33,
        178,  28, 179,   9,   1, 180,  35, 181, 182,  54,   6,  58,   2,
         28,  59,  36,  27, 183,  35, 184, 185,  37,  60,  35, 186,   1,
        187, 188,  61,   1, 189, 190,  62,   3, 191,  63,  34, 192, 193,
         64, 194,  38, 195,   1, 196, 197, 198, 199

In [123]:
y

array([[ 7,  2,  8,  9, 10,  3, 11, 12, 13, 14,  1, 15,  2, 16, 17, 18,
         1, 19, 20, 21, 22, 23, 24,  4, 25,  5, 26, 27,  6, 28, 29, 30,
        31, 32, 33, 34, 35, 36, 37, 38,  1, 39, 40, 41,  4,  6,  5, 42,
        43, 44,  3, 45, 46]])

In [124]:
Y_voc

47

In [126]:

from keras import backend as K 
K.clear_session()  #Resets all state generated by Keras

latent_dim = 100
embedding_dim = 100

# Encoder
encoder_inputs = Input(shape=(x.shape[1],))

# TODO: understand how embedding works here; is this our own trained embedding? maybe we should just use word2vec
#embedding layer
enc_emb =  Embedding(X_voc, embedding_dim,trainable=True)(encoder_inputs)

#encoder lstm
# TODO: why are we not using an activation function? default is none, only for recurrent activation the default is sigmoid
# TODO: why do we need the encoder_outputs? only for attention probably
encoder_lstm = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

#Setting up the Decoder using 'encoder_states' as initial state
# TODO: figure out why the shape is None? because we also use it later for our decoding when we only give one
decoder_inputs = Input(shape=((y.shape[1] - 1),))

#Embedding layer
dec_emb_layer = Embedding(Y_voc, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
# TODO: is the LSTM bidirectional? why do we even need those two states?
# TODO: why do the graphs say (None, 256) where 256 is the latent_dim (the length of the state vectors); shouldn't it be clear that it is (1, 256)?
decoder_outputs ,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb ,initial_state=[state_h, state_c])

# #Attention layer; removed for now
# attn_layer = AttentionLayer(name='attention_layer')
# attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# #Concating Attention input and Decoder LSTM output
# decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#Dense layer
# TODO: figure out what TimeDistributed does
decoder_dense =  TimeDistributed(Dense(units = Y_voc, activation='softmax'))
decoder_outputs = decoder_dense(decoder_outputs)

#Defining the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()

#Visualize the Model
from tensorflow.keras.utils import plot_model

plot_model(model, to_file='training_model_plot.png', show_shapes=True, show_layer_names=True)

# TODO: understand this
#Adding Metrics
model.compile(optimizer='rmsprop' , loss='sparse_categorical_crossentropy' , metrics=['accuracy'])

#Adding Callback
# TODO: how exactly does this work? is there an internal mapping of the string "val_loss" to running a test on the specified validation_data?
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1)

# Commented out IPython magic to ensure Python compatibility.
#Training the Model
# %tensorflow_version 1.x
# indexing is clear, removing the eos token for the decoder inpu
# ts and removing the sos token for the decoder outputs
# TODO: think about how exactly this is working with calculating the loss etc., maybe that's the problem
# TODO: fit only on one example and check if model actually works
# this creates the third dimension we need to compare to the generated output of the decoder; basically creates "one-hot encoding" to generate the probability distribution
history = model.fit(
    x = [x,y[:,:-1]], 
    y = y.reshape(y.shape[0], y.shape[1], 1)[:, 1:, :],
    epochs=200,
    batch_size = 5
)




Total params: 219,347
Trainable params: 219,347
Non-trainable params: 0
__________________________________________________________________________________________________
Epoch 1/200
1/1 [==============================] - 8s 8s/step - loss: 3.8499 - accuracy: 0.0000e+00
Epoch 2/200
1/1 [==============================] - 1s 982ms/step - loss: 3.8366 - accuracy: 0.0962
Epoch 3/200
1/1 [==============================] - 1s 949ms/step - loss: 3.8238 - accuracy: 0.1154
Epoch 4/200
1/1 [==============================] - 1s 981ms/step - loss: 3.8062 - accuracy: 0.0962
Epoch 5/200
1/1 [==============================] - 1s 937ms/step - loss: 3.7255 - accuracy: 0.0577
Epoch 6/200
1/1 [==============================] - 1s 944ms/step - loss: 3.6189 - accuracy: 0.0577
Epoch 7/200
1/1 [==============================] - 1s 959ms/step - loss: 3.4954 - accuracy: 0.0769
Epoch 8/200
1/1 [==============================] - 1s 926ms/step - loss: 3.3753 - accuracy: 0.0769
Epoch 9/200
1/1 [===================

In [136]:
#Building Dictionary for Source Vocabulary
# TODO: what is happening here? we are just saving the vocabularies of the tokenizers (index -> word or word -> index maps)
target_index_word=Y_tokenizer.index_word 
source_index_word=X_tokenizer.index_word 
target_word_index=Y_tokenizer.word_index

#Testing phase
#Encoding the input sequence to get the feature vector
# TODO: is this what links this model to the trained model? that we use the same encoder_inputs Input object?
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

#Decoder setup
#These tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
# TODO: remove this, probably only used for attention
# decoder_hidden_state_input = Input(shape=(max_text_len,latent_dim))

#Getting the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs) 

#Setting the initial states to the states from the previous time step for better prediction
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

# TODO: remove this
# #Attention inference
# attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
# decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

#Adding Dense softmax layer to generate proability distribution over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_outputs2) 

#Final Decoder model
# TODO: how is this all linked to what we trained before? where does the parameter sharing happen?
# TODO: why are we appending the lists instead of just writing it in one list
decoder_model = Model(
    [decoder_inputs] + [decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

#Function defining the implementation of inference process
def decode_sequence(input_seq):
    #Encoding the input as state vectors
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    #Generating empty target sequence of length 1
    target_seq = np.zeros((1,1))
    
    #Populating the first word of target sequence with the start word
    target_seq[0, 0] = target_word_index['sos']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_h, e_c])

        #Sampling a token
        # TODO: understand the indexing: why -1 instead of 0 as well? shouldn't the output tokens be of shape (1, 1, num_words)?
        # TODO: I think the neurons in the dense layer start at 0 but our dictionary starts at 1, despite a one-on-one mapping from
        # dictionary index numbers to neurons if I understand correctly; doesn't make sense to make, programm will crash if arg_max returns zero
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = target_index_word[sampled_token_index]
        
        if(sampled_token!='eos'):
            decoded_sentence += ' '+sampled_token

        #Exit condition: either hit max length or find stop word
        # TODO: it used to be >= (max_abstract_len - 1); why? doesn't make sense to me
        MAX_ABSTRACT_LEN = 60
        if (sampled_token == 'eos'  or len(decoded_sentence.split()) == MAX_ABSTRACT_LEN):
            stop_condition = True

        #Updating the target sequence (of length 1)
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        #Updating internal states
        e_h, e_c = h, c

    return decoded_sentence

#Functions to convert an integer sequence to a word sequence for summary as well as reviews 
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sos']) and i!=target_word_index['eos']):
            newString=newString+target_index_word[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+source_index_word[i]+' '
    return newString

#Summaries generated by the model

print("Predicted summary:",decode_sequence(x))
print("\n")

Predicted summary:  an injured officer says he was shot while trying to help an elderly man move to safetylapd provides timeline for welfare check at suspect s homesuspect paul ciancia is in critical condition and could face the death penaltypolice responded to family concern arrived at ciancia s apartment shortly after he left




In [1070]:
df = pd.DataFrame((cleaned_text, stories[]))

SyntaxError: invalid syntax (2442190599.py, line 1)

In [1071]:
df['abstract'] = df['abstract'].apply(lambda x : 'sostok '+ x + ' eostok')

#Splitting the Dataset twice to get 80% training data, 10% of validation data and 10% of test data
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val=train_test_split(np.array(df['article']),np.array(df['abstract']),test_size=0.2,random_state=0,shuffle=True)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size = 0.5, random_state = 0, shuffle = True)


NameError: name 'df' is not defined

In [1072]:
vocab_size =200
src_txt_length =200
sum_txt_length = 200
# encoder input model
inputs = Input(shape=(src_txt_length,))
encoder1 = Embedding(vocab_size, 128)(inputs)
encoder2 = LSTM(128)(encoder1)
encoder3 = RepeatVector(sum_txt_length)(encoder2)
# decoder output model
decoder1 = LSTM(128, return_sequences=True)(encoder3)
outputs = TimeDistributed(Dense(vocab_size, activation='softmax'))(decoder1)
# tie it together
model = Model(inputs=inputs, outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [1073]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

NameError: name 'nn' is not defined

In [1074]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

NameError: name 'nn' is not defined

In [1075]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

NameError: name 'nn' is not defined